In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.genmod.generalized_linear_model import SET_USE_BIC_LLF
from scipy.stats import nbinom
import random
import warnings
from statsmodels.tools.sm_exceptions import PerfectSeparationWarning

# Import custom modules and functions
from generate_hnb import generate_hnb
from generate_ZI import generate_ZI
from AIC_BIC import calculate_aic_bic

# Import model classes
from models.ZKIHurdlePoisson import ZKHurdlePoisson
from models.ZKIHurdleNB import ZKHurdleNB
from models.ZINB import ZINB_EM, predict_mean as ZINB_pred_mean
from models.ZIP import ZIP_EM, predict_mean as ZIP_pred_mean
from models.ZKINB import ZkINB_EM
from models.ZKIP import ZKIP_EM
from models.ZkICMP import ZkICMP

from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
# warnings.filterwarnings('ignore', module='statsmodels')
warnings.filterwarnings('ignore', category=PerfectSeparationWarning)

class ModelEvaluator:
    """Class to evaluate and compare different count data models."""
    
    def __init__(self, X_train, X_test, y_train, y_test):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.n_train = X_train.shape[0]
        self.results = {}
    
    def evaluate_model(self, model_name, y_pred_test, y_pred_train, llf, k_params):
        """Calculate evaluation metrics for a model."""
        metrics = {
            'mse_test': mean_squared_error(self.y_test, y_pred_test),
            'mae_test': mean_absolute_error(self.y_test, y_pred_test),
            'r2_test': r2_score(self.y_test, y_pred_test),
            'mse_train': mean_squared_error(self.y_train, y_pred_train),
            'mae_train': mean_absolute_error(self.y_train, y_pred_train),
            'r2_train': r2_score(self.y_train, y_pred_train),
            'llf': llf,
            'aic': calculate_aic_bic(self.n_train, llf, k_params)[0],
            'bic': calculate_aic_bic(self.n_train, llf, k_params)[1]
        }
        self.results[model_name] = metrics
        return metrics
    
class ModelEvaluatorML:
    def __init__(self, X_train, X_test, y_train, y_test):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.n_train = X_train.shape[0]
        self.results = {}
    
    def evaluate_model(self, model_name, y_pred_test, y_pred_train):
        """Calculate evaluation metrics for a model."""
        metrics = {
            'mse_test': mean_squared_error(self.y_test, y_pred_test),
            'mae_test': mean_absolute_error(self.y_test, y_pred_test),
            'r2_test': r2_score(self.y_test, y_pred_test),
            'mse_train': mean_squared_error(self.y_train, y_pred_train),
            'mae_train': mean_absolute_error(self.y_train, y_pred_train),
            'r2_train': r2_score(self.y_train, y_pred_train)
        }
        self.results[model_name] = metrics
        return metrics

def define_model_parameters():
    """Define parameter counts for different models."""
    return {
        'poisson': 2,
        'negative_binomial': 3,  # if r known (2)
        'zk_hurdle_poisson': 4,  # 2+2
        'zk_hurdle_nb': 5, # 2+2+1 , 1 for alpha
        'zinb': 5,  # if r known (2+2+1)
        'zip': 4,  # 2+2
        'zkinb': 7,  # if r known (2+2+2+1)
        'zkip': 6,  # 2+2+2
        'zkicmp': 7  # 2+2+2+1
    }

def fit_models(X_train, X_test, y_train, y_test, k):
    # Configuration
    SET_USE_BIC_LLF(True)
    
    # Get parameter counts
    param_counts = define_model_parameters()
    
    # Initialize evaluator
    evaluator = ModelEvaluator(X_train, X_test, y_train, y_test)
    
    # 1. Poisson Model
    #print("Fitting Poisson model...")
    try:
        poisson_model = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
        poisson_pred = poisson_model.predict(X_test)
        poisson_pred_train = poisson_model.predict(X_train)
        evaluator.evaluate_model(
            'pois', poisson_pred, poisson_pred_train, poisson_model.llf, param_counts['poisson']
        )
    except Exception as e:
        print(e)
    
    # 2. Negative Binomial Model
    #print("Fitting Negative Binomial model...")
    try:
        nb_model = sm.NegativeBinomial(y_train, X_train).fit(disp=0)
        nb_pred = nb_model.predict(X_test)
        nb_pred_train = nb_model.predict(X_train)
        evaluator.evaluate_model(
            'nb', nb_pred, nb_pred_train, nb_model.llf, param_counts['negative_binomial']
        )
    except Exception as e:
        print(e)
    
    # 3. Zero-K Inflated Poisson Hurdle Model
    #print("Fitting Zero-K Inflated Poisson Hurdle model...")
    try:
        zkihurdle_model = ZKHurdlePoisson(k)
        zkihurdle_res = zkihurdle_model.fit(X_train, y_train)
        zkihurdle_ll = zkihurdle_model.loglikelihood(X_train, y_train)
        zkihurdle_pred = zkihurdle_model.predict_mean(X_test)
        zkihurdle_pred_train = zkihurdle_model.predict_mean(X_train)
        evaluator.evaluate_model(
            'zk_h_p', zkihurdle_pred, zkihurdle_pred_train, zkihurdle_ll, param_counts['zk_hurdle_poisson']
        )
    except Exception as e:
        print(e)

    # 5. ZKIHurdleNB Model
    try:
        zkihurdlenb_modle=ZKHurdleNB(k, alpha=0.1)
        zkihurdlenb_modle.fit(X_train, y_train)
        zkihurdlenb_pred=zkihurdlenb_modle.predict_mean(X_test)
        zkihurdlenb_pred_train=zkihurdlenb_modle.predict_mean(X_train)
        zkihurdlenb_ll=zkihurdlenb_modle.loglikelihood(X_train, y_train)
        evaluator.evaluate_model(
            'zk_h_nb', zkihurdlenb_pred, zkihurdlenb_pred_train, zkihurdlenb_ll, param_counts['zk_hurdle_nb']
        )
    except Exception as e:
        print(e)

    # 4. ZINB Model
    #print("Fitting ZINB model...")
    try:
        alpha = 1/10
        beta, gamma, zinb_ll = ZINB_EM(y_train.values, X_train.values, X_train.values, alpha)
        zinb_pred = ZINB_pred_mean(X_test.values, X_test.values, beta, gamma)
        zinb_pred_train = ZINB_pred_mean(X_train.values, X_train.values, beta, gamma)
        evaluator.evaluate_model(
            'zinb', zinb_pred, zinb_pred_train, zinb_ll, param_counts['zinb']
        )
    except Exception as e:
        print(e)
    
    # 5. ZIP Model
    #print("Fitting ZIP model...")
    try:
        beta, gamma, zip_ll = ZIP_EM(y_train.values, X_train.values, X_train.values)
        zip_pred = ZIP_pred_mean(X_test.values, X_test.values, beta, gamma)
        zip_pred_train = ZIP_pred_mean(X_train.values, X_train.values, beta, gamma)   
        evaluator.evaluate_model(
            'zip', zip_pred, zip_pred_train, zip_ll, param_counts['zip']
        )
    except Exception as e:
        print(e)

    # 6. ZKINB Model
    #print("Fitting ZKINB model...")
    try:
        zkinb_model = ZkINB_EM()
        zkinb_res = zkinb_model.fit_em(y_train.values, X_train.values, X_train.values, k)
        zkinb_pred = zkinb_model.predict(X_test.values, X_test.values)
        zkinb_pred_train = zkinb_model.predict(X_train.values, X_train.values)
        evaluator.evaluate_model(
            'zkinb', zkinb_pred, zkinb_pred_train, zkinb_res['final_loglik'], param_counts['zkinb']
        )
    except Exception as e:
        print(e)

    # 7. ZKIP Model
    #print("Fitting ZKIP model...")
    try:
        zkip_model = ZKIP_EM(k_inflated=k)
        zkip_res = zkip_model.fit(X_train.values, y_train.values)
        zkip_pred = zkip_model.predict_expected(X_test.values)
        zkip_pred_train = zkip_model.predict_expected(X_train.values)
        evaluator.evaluate_model(
            'zkip', zkip_pred, zkip_pred_train, zkip_res.final_loglik, param_counts['zkip']
        )
    except Exception as e:
        print(e)

    '''
    # 8. ZkICMP Model
    print("Fitting ZkICMP model...")
    zkicmp_model = ZkICMP(k=k)
    zkicmp_res = zkicmp_model.fit(X_train.values, y_train.values)
    pred_results = zkicmp_model.predict(X_test.values)
    _, _, zkicmp_pred, _ = pred_results
    evaluator.evaluate_model(
        'zkicmp', zkicmp_pred, -zkicmp_res.final_loglik, param_counts['zkicmp']
    )
    '''
    
    results = {}
    for model_name, metrics in evaluator.results.items():
        results[f'{model_name.upper()}_MSE_test'] = metrics['mse_test']
        results[f'{model_name.upper()}_MAE_test'] = metrics['mae_test']
        results[f'{model_name.upper()}_R2_test'] = metrics['r2_test']
        results[f'{model_name.upper()}_MSE_train'] = metrics['mse_train']
        results[f'{model_name.upper()}_MAE_train'] = metrics['mae_train']
        results[f'{model_name.upper()}_R2_train'] = metrics['r2_train']
        results[f'{model_name.upper()}_LLF'] = metrics['llf']
        results[f'{model_name.upper()}_AIC'] = metrics['aic']
        results[f'{model_name.upper()}_BIC'] = metrics['bic']
    
    return results

def fit_ML_models(X_train, X_test, y_train, y_test):
    # Initialize evaluator
    evaluator = ModelEvaluatorML(X_train, X_test, y_train, y_test)


    # RandomForestRegressor
    try:
        rf = RandomForestRegressor(
                n_estimators=300,
                criterion="poisson",
                min_samples_leaf=20,
                n_jobs=-1,
                random_state=42
            )
        rf.fit(X_train, y_train)
        y_pred_rf = rf.predict(X_test)
        y_pred_train_rf = rf.predict(X_train)
        evaluator. evaluate_model('RF', y_pred_rf, y_pred_train_rf)
    except Exception as e:
        print(e)
    
    # HistGradientBoostingRegressor with Poisson loss
    try:
        poisson_GB = HistGradientBoostingRegressor(
                    loss='poisson',        # This is the key - uses Poisson likelihood
                    random_state=42,
                    max_iter=100,
                    learning_rate=0.1,
                    max_depth=6,
                    min_samples_leaf=20    # Good for count data to prevent overfitting
                )
        poisson_GB.fit(X_train, y_train)
        y_pred_gb = poisson_GB.predict(X_test)
        y_pred_train_gb = poisson_GB.predict(X_train)
        evaluator. evaluate_model('GB', y_pred_gb, y_pred_train_gb)
    except Exception as e:
        print(e)

    results = {}
    for model_name, metrics in evaluator.results.items():
        results[f'{model_name.upper()}_MSE_test'] = metrics['mse_test']
        results[f'{model_name.upper()}_MAE_test'] = metrics['mae_test']
        results[f'{model_name.upper()}_R2_test'] = metrics['r2_test']
        results[f'{model_name.upper()}_MSE_train'] = metrics['mse_train']
        results[f'{model_name.upper()}_MAE_train'] = metrics['mae_train']
        results[f'{model_name.upper()}_R2_train'] = metrics['r2_train']
    
    return results

def check_for_nan(X_train, X_test, y_train, y_test):
    """Check for NaN values in data."""
    checks = {
        'X_train_nan': np.any(np.isnan(X_train.values)),
        'X_test_nan': np.any(np.isnan(X_test.values)),
        'y_train_nan': np.any(np.isnan(y_train.values)),
        'y_test_nan': np.any(np.isnan(y_test.values)),
        'X_train_inf': np.any(np.isinf(X_train.values)),
        'y_train_inf': np.any(np.isinf(y_train.values))
    }
    return checks

def main(n, rep, gen_m):

    k = 3
    test_size=0.3
    # Split data indexes
    train_ind, test_ind = train_test_split(
        np.arange(0, n), 
        test_size=test_size, random_state=42
    )

    results = []
    results_r2_minus = []
    problmatic_para = []
    err=[]
    random.seed(42)
    for replication in range(0, rep):
        for beta in np.linspace(-2, 2, 5):  
            for gamma in np.linspace(-2, 2, 5):
                for alpha in np.linspace(-2, 2, 5):
                    # print(f"\nIteration: beta={beta:.2f}, gamma={gamma:.2f}, alpha={alpha:.2f}")
                    
                    """Generate and prepare the dataset for modeling."""
                    # Generate data - using the parameters from the loops   
                    if gen_m=='hnb':                 
                        df = generate_hnb(
                                n=n, k=k, beta0=beta, beta1=1, gamma0=gamma, gamma1=0.3,
                                alpha0=alpha, alpha1=1, r=10, cov_type="nbinary"
                            )
                    if gen_m=='zi':
                        df = generate_ZI(
                                n=n, k=k, beta0=beta, beta1=1, gamma0=gamma, gamma1=0.3,
                                alpha0=alpha, alpha1=1, r=10, cov_type="nbinary"
                            )
                        # Prepare features and target
                    X = df.loc[:, 'x'].values.reshape(-1, 1)
                    y_target = df.loc[:, 'y'].values
                        
                    # Add intercept
                    X = sm.add_constant(X)
                    X = pd.DataFrame(X, columns=['intercept', 'x'])
                    y_target = pd.Series(y_target, name='y')
                        
                    X_train = X.iloc[train_ind, :]
                    X_test = X.iloc[test_ind, :]
                    y_train = y_target.iloc[train_ind]
                    y_test = y_target.iloc[test_ind]

                    p_0 = (y_train == 0).mean()
                    p_k = (y_train == k).mean()
                    p_p = max(1 - p_0 - p_k, 0)
                    # Calculate statistics
                    y_mean = y_train.mean()
                    y_std = y_train.std()
                    n_unique = len(pd.Series(y_train).value_counts())
                    
                    checks = check_for_nan(X_train, X_test, y_train, y_test)
                    if any(checks.values()):
                        print(f"NaN/Inf found: {checks}")
                        print(f"y_mean: {y_mean}, y_std: {y_std}")
                        print(f"y_std^2 - y_mean: {y_std**2 - y_mean}")
                        continue  

                    if not min(p_0, p_k, p_p) >= 3/(n*(1-test_size)):
                        problmatic_para.append({
                                'beta': beta,
                                'gamma': gamma,
                                'alpha': alpha,
                                'p_0': p_0,
                                'p_k': p_k,
                                'p_p': p_p,
                                'y_mean': y_mean,
                                'y_std': y_std,
                                'n_unique': n_unique
                            })
                                    
                    else:
                        # Estimate r for negative binomial
                        if y_std**2 > y_mean:
                            r_hat = y_mean**2 / max(y_std**2 - y_mean, 1e-9)
                            p0_nb = nbinom.pmf(0, r_hat, r_hat/(r_hat + y_mean))
                            pk_nb = nbinom.pmf(k, r_hat, r_hat/(r_hat + y_mean))
                            zero_inflated = p_0 > p0_nb 
                            k_inflated = p_k > pk_nb 
                            # Fit models
                            try:
                                model_results = fit_models(X_train, X_test, y_train, y_test, k)
                                ml_model_results=fit_ML_models(X_train, X_test, y_train, y_test)
                                results.append({
                                            'beta': beta,
                                            'gamma': gamma,
                                            'alpha': alpha,
                                            'p_0': p_0,
                                            'p_k': p_k,
                                            'p_p': p_p,
                                            'y_mean': y_mean,
                                            'y_std': y_std,
                                            'n_unique': n_unique,
                                            'r_hat': r_hat,
                                            'p0_nb': p0_nb,
                                            'pk_nb': pk_nb,
                                            'zero_inflated': zero_inflated,
                                            'k_inflated': k_inflated,
                                    } | model_results
                                      | ml_model_results)
                            except Exception as e:
                                print(e)
                                err.append({
                                    'beta': beta,
                                    'gamma': gamma,
                                    'alpha': alpha,
                                    'p_0': p_0,
                                    'p_k': p_k,
                                    'p_p': p_p,
                                    'y_mean': y_mean,
                                    'y_std': y_std,
                                    'n_unique': n_unique
                                })
                        else:
                            r_hat = np.nan
                            p0_nb = np.nan
                            pk_nb = np.nan
                            results_r2_minus.append({
                                            'beta': beta,
                                            'gamma': gamma,
                                            'alpha': alpha,
                                            'p_0': p_0,
                                            'p_k': p_k,
                                            'p_p': p_p,
                                            'y_mean': y_mean,
                                            'y_std': y_std,
                                            'n_unique': n_unique,
                                        })
        
    return pd.DataFrame(results), pd.DataFrame(results_r2_minus), pd.DataFrame(problmatic_para), pd.DataFrame(err)

#if __name__ == "__main__":
#    results_H, results_r2_minus_H, problmatic_para_H, err= main(300, 10)

In [42]:
n=100
rep=50
gen_m='zi' # hnb or zi
results, results_r2_minus, problmatic_para, err= main(n, rep, gen_m)
results.to_csv(f'results/results_{gen_m}_n={n}_rep={rep}.csv')
results_r2_minus.to_csv(f'results/results_r2_minus_{gen_m}_n={n}_rep={rep}.csv')
problmatic_para.to_csv(f'results/problmatic_para_{gen_m}_n={n}_rep={rep}.csv')
err.to_csv(f'results/err_{gen_m}_n={n}_rep={rep}.csv')

/Users/pasindumadusanka/Documents/GitHub/ResearchProject/ResearchProjectMain/Final Simulation/models/ZINB.py:43: RuntimeWarning: invalid value encountered in divide
  z = eta + (y - p) / (p * (1 - p))
/Users/pasindumadusanka/Documents/GitHub/ResearchProject/ResearchProjectMain/Final Simulation/models/ZIP.py:37: RuntimeWarning: invalid value encountered in divide
  z = eta + (y - p) / (p * (1 - p))


Input contains NaN.
Input contains NaN.


/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/pasindumadusanka/Documents/GitHub/ResearchProject/ResearchProjectMain/Final Simulation/models/ZKIP.py:72: RuntimeWarning: divide by zero encountered in log
  delta_new = np.log(np.sum(z2_hat) / np.sum(z3_hat)) if np.sum(z3_hat) > 0 else self.delta
/Users/pasindumadusanka/Documents/GitHub/ResearchProject/ResearchProjectMain/Final Simulation/models/ZKIP.py:122: RuntimeWarning: invalid value encountered in scalar subtract
  delta_diff = np.abs(delta_new - self.delta)
/Users/pasindumadusanka/Documents/GitHub/ResearchProject/ResearchProjectMain/Final Simulation/models/ZINB.py:43: RuntimeWarning: divide by zero encountered in divide
  z = eta + (y - p) / (p * (1 - p))
/Users/pasindumadusanka/Documents/GitHub/ResearchProjec

Input contains NaN.
Input contains NaN.


/Users/pasindumadusanka/Documents/GitHub/ResearchProject/ResearchProjectMain/Final Simulation/models/ZINB.py:43: RuntimeWarning: invalid value encountered in divide
  z = eta + (y - p) / (p * (1 - p))


Input contains NaN.


In [43]:
n=300
rep=50
gen_m='zi' # hnb or zi
results, results_r2_minus, problmatic_para, err= main(n, rep, gen_m)
results.to_csv(f'results/results_{gen_m}_n={n}_rep={rep}.csv')
results_r2_minus.to_csv(f'results/results_r2_minus_{gen_m}_n={n}_rep={rep}.csv')
problmatic_para.to_csv(f'results/problmatic_para_{gen_m}_n={n}_rep={rep}.csv')
err.to_csv(f'results/err_{gen_m}_n={n}_rep={rep}.csv')

In [ ]:
n=1000
rep=50
gen_m='zi' # hnb or zi
results, results_r2_minus, problmatic_para, err= main(n, rep, gen_m)
results.to_csv(f'results/results_{gen_m}_n={n}_rep={rep}.csv')
results_r2_minus.to_csv(f'results/results_r2_minus_{gen_m}_n={n}_rep={rep}.csv')
problmatic_para.to_csv(f'results/problmatic_para_{gen_m}_n={n}_rep={rep}.csv')
err.to_csv(f'results/err_{gen_m}_n={n}_rep={rep}.csv')

In [ ]:
n=5000
rep=50
gen_m='zi' # hnb or zi
results, results_r2_minus, problmatic_para, err= main(n, rep, gen_m)
results.to_csv(f'results/results_{gen_m}_n={n}_rep={rep}.csv')
results_r2_minus.to_csv(f'results/results_r2_minus_{gen_m}_n={n}_rep={rep}.csv')
problmatic_para.to_csv(f'results/problmatic_para_{gen_m}_n={n}_rep={rep}.csv')
err.to_csv(f'results/err_{gen_m}_n={n}_rep={rep}.csv')

In [ ]:
n=10000
rep=50
gen_m='zi' # hnb or zi
results, results_r2_minus, problmatic_para, err= main(n, rep, gen_m)
results.to_csv(f'results/results_{gen_m}_n={n}_rep={rep}.csv')
results_r2_minus.to_csv(f'results/results_r2_minus_{gen_m}_n={n}_rep={rep}.csv')
problmatic_para.to_csv(f'results/problmatic_para_{gen_m}_n={n}_rep={rep}.csv')
err.to_csv(f'results/err_{gen_m}_n={n}_rep={rep}.csv')

In [36]:
(results.isna().sum()!=0).sum()

0

In [37]:
results.columns

Index(['beta', 'gamma', 'alpha', 'p_0', 'p_k', 'p_p', 'y_mean', 'y_std',
       'n_unique', 'r_hat', 'p0_nb', 'pk_nb', 'zero_inflated', 'k_inflated',
       'POIS_MSE_test', 'POIS_MAE_test', 'POIS_R2_test', 'POIS_MSE_train',
       'POIS_MAE_train', 'POIS_R2_train', 'POIS_LLF', 'POIS_AIC', 'POIS_BIC',
       'NB_MSE_test', 'NB_MAE_test', 'NB_R2_test', 'NB_MSE_train',
       'NB_MAE_train', 'NB_R2_train', 'NB_LLF', 'NB_AIC', 'NB_BIC',
       'ZK_H_P_MSE_test', 'ZK_H_P_MAE_test', 'ZK_H_P_R2_test',
       'ZK_H_P_MSE_train', 'ZK_H_P_MAE_train', 'ZK_H_P_R2_train', 'ZK_H_P_LLF',
       'ZK_H_P_AIC', 'ZK_H_P_BIC', 'ZK_H_NB_MSE_test', 'ZK_H_NB_MAE_test',
       'ZK_H_NB_R2_test', 'ZK_H_NB_MSE_train', 'ZK_H_NB_MAE_train',
       'ZK_H_NB_R2_train', 'ZK_H_NB_LLF', 'ZK_H_NB_AIC', 'ZK_H_NB_BIC',
       'ZINB_MSE_test', 'ZINB_MAE_test', 'ZINB_R2_test', 'ZINB_MSE_train',
       'ZINB_MAE_train', 'ZINB_R2_train', 'ZINB_LLF', 'ZINB_AIC', 'ZINB_BIC',
       'ZIP_MSE_test', 'ZIP_MAE_test', 'ZIP_R2_test

In [38]:
err

""
